# Convolutional Neural Networks¶

Nick Chao

#### Dataset Selection

Select a dataset identically to the way you selected for the lab one (i.e., table data). You are not required to use the same dataset that you used in the past, but you are encouraged. You must identify a classification task from the dataset that contains three or more classes to predict. That is it cannot be a binary classification; it must be multi-class prediction. 

### Preparation (30 points total)  

[15 points] Choose and explain what metric(s) you will use to evaluate your algorithm’s performance. You should give a detailed argument for why this (these) metric(s) are appropriate on your data. That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task). Please note: rarely is accuracy the best evaluation metric to use. Think deeply about an appropriate measure of performance.

[15 points] Choose the method you will use for dividing your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Convince me that your cross validation method is a realistic mirroring of how an algorithm would be used in practice. 

### Modeling (60 points total)
[15 points] Create a convolutional neural network to use on your data using Keras. Setup the training to use data expansion in Keras. Explain why the chosen data expansion techniques are appropriate for your dataset. 

[20 points] Investigate at least two different convolutional network architectures (and investigate changing some parameters of each architecture). Use the method of cross validation and evaluation metric that you argued for at the beginning of the lab. Visualize the performance of the training and validation sets per iteration (use the "history" parameter of Keras).

[15 points] Visualize the final results of the CNNs and interpret the performance. Use proper statistics as appropriate, especially for comparing models. 

[10 points] Compare the performance of your convolutional network to a standard multi-layer perceptron (MLP) using the receiver operating characteristic and area under the curve. Use proper statistical comparison techniques.  

### Exceptional Work (10 points total)

You have free reign to provide additional analyses. 
One idea (required for 7000 level students): Use transfer learning to pre-train the weights of your initial layers of your CNN. Compare the performance when using transfer learning to training from scratch in terms of classification performance. 

### Business Understanding

In this lab we will be dealing with image data. I've decided to use Cidar-10 small images as there are about 60,000 (32x32) images with the following labels:

- airplane
- automobile
- bird
- cat
- deer
- dog
- frog
- horse
- ship
- truck

The goal with this lab is to create a model that can predict as well as possible according to the F1 Score. A succcessful model would have great applications with future technology as it can perform image recongnition with multiple classes. If it is possible to deploy a model that has a strong F1 score then it could be used for people large, expensive computationally expensive computers. 

For this to be an effective application, I would ideally want an F1 score of around 90% so that the majority of images can be classified correctly without the need for a human to intervene that much. However, this may not be possible so my goal would be around 70-75% but with high catagorical accuracy and without having to train the model for days. 

### Dependencies

In [27]:
import os
import keras
import numpy as np
import seaborn as sns
import _pickle as pickle

from matplotlib import pyplot as plt

from keras import backend as K
from keras.datasets import cifar10
from keras.models import Sequential, Input, Model
from keras.layers import Reshape, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, UpSampling2D
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
# from keras.layers.normalization import BatchNormalization
# from keras.callbacks import EarlyStopping
# from keras.constraints import maxnorm
# from keras.optimizers import SGD
# from keras.layers.convolutional import Conv2D, MaxPooling2D
# from keras.utils import np_utils

# from scipy import interp

from sklearn import metrics as mt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
# from sklearn.pipeline import Pipeline
# from skimage.io import imshow
# from skimage.feature import daisy


%matplotlib inline

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

## Preparation

In [29]:
# From https://keras.io/datasets/
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [30]:
NUM_CLASSES = 10
Y_train_ohe = keras.utils.to_categorical(Y_train, NUM_CLASSES)
Y_test_ohe = keras.utils.to_categorical(Y_test, NUM_CLASSES)

In [31]:
datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(X_train)